In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hadoop fs -rm -f -r tmp;

Deleted tmp


In [3]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 13.417 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.954 seconds


In [4]:
%%hive
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.484 seconds


In [5]:
%%hive
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
    SELECT c1, CONCAT_WS(':', COLLECT_LIST(UPPER(letra))) AS LM
    FROM
        tbl0 LATERAL VIEW explode(c5) tbl10 AS letra
    GROUP BY c1;
--SELECT * 
--FROM salida 
--LIMIT 5;     

DROP TABLE IF EXISTS salida;
OK
Time taken: 0.122 seconds
CREATE TABLE salida
AS
    SELECT c1, CONCAT_WS(':', COLLECT_LIST(UPPER(letra))) AS LM
    FROM
        tbl0 LATERAL VIEW explode(c5) tbl10 AS letra
    GROUP BY c1;
Query ID = root_20200119201613_240b0d01-c0b3-44ad-b6a2-6f15ab8ab791
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579456111145_0011, Tracking URL = http://a902e746193c:8088/proxy/application_1579456111145_0011/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579456111145_0011
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-19 20:16:27,950 Stage-1 map

In [6]:
%%hive
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    LM
FROM
    salida;

INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    LM
FROM
    salida;
Query ID = root_20200119201653_d666d23e-033e-4c40-91ec-31647950f876
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579456111145_0012, Tracking URL = http://a902e746193c:8088/proxy/application_1579456111145_0012/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579456111145_0012
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-19 20:17:07,422 Stage-1 map = 0%,  reduce = 0%
2020-01-19 20:17:12,926 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.45 sec
MapReduce Total cumulative CPU time: 1 seconds 450 msec
Ended Job = job_1579456111145_0012
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/000000

In [7]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [8]:
%hive_quit